In [1]:
from selenium import webdriver
from selenium.webdriver.remote.errorhandler import NoSuchElementException
from datetime import datetime as dt
import yaml

In [2]:
def athena_driver(username, password, headless=True, download_dir=None):
    options = webdriver.ChromeOptions()
    options.add_argument("--incognito")
    options.add_argument('window-size=1680x900')
    options.headless = headless
    if download_dir:
        options.add_experimental_option('prefs', {'download.default_directory': download_dir})
    driver = webdriver.Chrome(executable_path='chromedriver.exe', options=options)
    driver.set_window_position(0,0)
    driver.set_window_size(1200, 1375)
    driver.set_page_load_timeout(15)
    
    driver.get("https://athena.ohdsi.org/")
    driver.find_element_by_xpath('/html/body/div/div/div[4]/div[2]/div/div[2]/div[2]/button').click()
    driver.find_element_by_xpath('/html/body/div/div/header/nav/div[3]/a').click()
    driver.find_element_by_xpath('/html/body/div/div/div[1]/div/div[2]/div/button').click()

    main_window = driver.current_window_handle
    for handle in driver.window_handles:
        if handle != main_window:
            popup = handle
            driver.switch_to.window(popup)

    driver.find_element_by_xpath('//*[@id="username"]').send_keys(username)
    driver.find_element_by_xpath('//*[@id="password"]').send_keys(password)
    driver.find_element_by_xpath('/html/body/div[1]/div/div/div[2]/div/form/section[3]/input[4]').click()
    driver.switch_to.window(driver.window_handles[0])

    return driver 

In [65]:
def request_new_vocabulary_file(driver):
    driver.get('https://athena.ohdsi.org/search-terms/start')
    driver.find_element_by_xpath('/html/body/div/div/header/nav/div[3]/div/a/div[2]').click()
    driver.find_element_by_xpath('//*[@id="app"]/div/header/nav/div[3]/div/div/div/a[1]').click()
    driver.find_element_by_xpath('//*[@id="app"]/div/header/nav/div[2]/a').click()
    
    # double click on select all to make sure no vocab is selected 
    driver.find_element_by_xpath('//*[@id="app"]/div/div[1]/div[2]/table/thead/tr/th[1]/label').click()
    driver.find_element_by_xpath('//*[@id="app"]/div/div[1]/div[2]/table/thead/tr/th[1]/label').click()

    # Select vocabularies
    checkboxes = driver.find_elements_by_css_selector("#app > div > div.at-vocabs > div.at-vocabularies > table > tbody > tr")
    with open('omop_vocabularies.yml') as f:
        vocabs = yaml.safe_load(f)
    
    for checkbox in checkboxes:
        cls = checkbox.find_element_by_class_name('at-vocabularies__code-td')
        if cls.text in vocabs:
            checkbox.click()
            print(f'Vocabulary {cls.text} was selected')
    
    datestamp = dt.today().strftime('%m_%d_%Y')
    driver.find_element_by_xpath('//*[@id="app"]/div/div[1]/div[1]/button').click()
    driver.find_element_by_xpath('//*[@id="app"]/div/div[1]/div[3]/div[2]/div/div[2]/form/div/div[1]/input').send_keys(f'vocabulary_5x_{datestamp}')
    
    # Request vocab file
    driver.find_element_by_xpath('//*[@id="app"]/div/div[1]/div[3]/div[2]/div/div[2]/form/div/div[2]/button[1]').click()

    #return to main page
    driver.get('https://athena.ohdsi.org/search-terms/start')

In [7]:
def download_vocabulary_file(driver, get_last=True, vocabulary_name=None, archive=True, restore=True):
    """Download most recent vocabulary file."""
    driver.get('https://athena.ohdsi.org/search-terms/start')
    driver.find_element_by_xpath('/html/body/div/div/header/nav/div[3]/div/a/div[2]').click()
    driver.find_element_by_xpath('//*[@id="app"]/div/header/nav/div[3]/div/div/div/a[1]').click()
    athena.find_element_by_xpath('//*[@id="app"]/div/header/nav/div[3]/div').click()
    buttons = driver.find_elements_by_class_name("react-sanfona-item")
    if get_last: 
        button = buttons[0]
        tool = button.find_element_by_class_name('ac-toolbar')
        status = tool.text.split()[-1]
        if status == 'DOWNLOADSHAREARCHIVE':
            print(tool.text)
            tool.click()
            if archive:
                driver.find_element_by_xpath('//*[@id="app"]/div/div[1]/div[2]/div[1]/div[1]/ul/li/div/button[3]').click()
        elif (tool.text == 'RESTORE') & restore is True:
            print('Restoring vocabulary, try again later')
            tool.click()
            # Wait 30min while file is restored.
            # If archive: archive.click()
#             driver.find_element_by_xpath('//*[@id="app"]/div/div[1]/div[2]/div[1]/div[1]/ul/li/div/button[3]').click()
        elif (tool.text == 'RESTORE') & restore is False:
            return 'Vocabulary file is archived. Set restore=True to restore and download.'
#         except NoSuchElementException: 
#             tool = button.find_element_by_class_name('ac-toolbar')
#             res = tool.text.split()[-1]
#             if res == 'PENDING': 
#                 print("Restoring vocabulary, try again later.")
        finally: 
            driver.get('https://athena.ohdsi.org/search-terms/start')

In [67]:
# Login to Athena
#TODO - get an account that everyone can access.
USERNAME = "yankuic@ufl.edu"
PASSWORD = "7DSrMx7JNTHaXEM"
athena = athena_driver(USERNAME, PASSWORD, headless=False)
request_new_vocabulary_file(athena)

Vocabulary SNOMED was selected
Vocabulary ICD9CM was selected
Vocabulary ICD9Proc was selected
Vocabulary CPT4 was selected
Vocabulary HCPCS was selected
Vocabulary LOINC was selected
Vocabulary NDFRT was selected
Vocabulary RxNorm was selected
Vocabulary NDC was selected
Vocabulary Gender was selected
Vocabulary Race was selected
Vocabulary CMS Place of Service was selected
Vocabulary ATC was selected
Vocabulary VA Product was selected
Vocabulary VA Class was selected
Vocabulary ICD10 was selected
Vocabulary ICD10PCS was selected
Vocabulary DRG was selected
Vocabulary MDC was selected
Vocabulary APC was selected
Vocabulary Revenue Code was selected
Vocabulary Ethnicity was selected
Vocabulary MeSH was selected
Vocabulary NUCC was selected
Vocabulary Medicare Specialty was selected
Vocabulary SPL was selected
Vocabulary GCN_SEQNO was selected
Vocabulary OPCS4 was selected
Vocabulary HES Specialty was selected
Vocabulary PCORNet was selected
Vocabulary Currency was selected
Vocabulary I

In [12]:
download_vocabulary_file(athena)

In [86]:
# athena.switch_to.window(athena.window_handles[0])
athena.get('https://athena.ohdsi.org/search-terms/start')
athena.find_element_by_xpath('/html/body/div/div/header/nav/div[3]/div/a/div[2]').click()
athena.find_element_by_xpath('//*[@id="app"]/div/header/nav/div[3]/div/div/div/a[1]').click()
athena.find_element_by_xpath('//*[@id="app"]/div/header/nav/div[3]/div').click()
buttons = athena.find_elements_by_class_name("react-sanfona-item")
last = True
archive = True
if last:
    button = buttons[0]
    tool = button.find_element_by_class_name('ac-toolbar')
    status = tool.text.split()[-1]
    if status == 'DOWNLOADSHAREARCHIVE': 
        athena.find_element_by_xpath('//*[@id="app"]/div/div[1]/div[2]/div[1]/div[1]/ul/li/div/button[1]').click()
        if archive:
            athena.find_element_by_xpath('//*[@id="app"]/div/div[1]/div[2]/div[1]/div[1]/ul/li/div/button[3]').click()
    elif status == 'ARCHIVEDRESTORE':
        print(status)

ARCHIVEDRESTORE


In [18]:
# Implement 30 min wait - it takes 15-20 min for ATHENA to process the download file. 

# Download vocabulary
driver.find_element_by_xpath('//*[@id="app"]/div/div[1]/div[2]/div[1]/div[1]/ul/li/div/button[1]').click()

if archive:
# Implement 5 min wait then archive the vocabulary
    driver.find_element_by_xpath('//*[@id="app"]/div/div[1]/div[2]/div[1]/div[1]/ul/li/div/button[3]').click()

In [18]:
# Unzip vocab zip file
!tar -xf C:\Users\yankuic\Downloads\vocabulary_download_v5_{3715153c-1726-4268-a5b3-c3a8a3921c43}_1617991037969.zip

In [ ]:
#app > div > div.at-download-history > div.react-sanfona > div:nth-child(1) > div.ac-toolbar > ul > li > div > span